# `qutomo`: A toolkit for fast and scalable QST 

We briefly overview some of the key functionalities of `qutomo`. We solve the following optimization problem for QST:

\begin{equation}\label{eq:obj}
\begin{aligned}
& \min_{\rho \in \mathbb{C}^{d \times d}}
& & f(\rho) := \tfrac{1}{2} \|\mathcal{A}(\rho) - y\|_2^2 \\
& \text{subject to}
& & \rho \succeq 0, ~\texttt{rank}(\rho) \leq r.
\end{aligned}
\end{equation}

The main idea is to factorize the density matrix $\rho = UU^\dagger$, which allows us to re-write the above optimization problem as:

\begin{equation}\label{eq:factobj}
\min_{U \in \mathbb{C}^{d \times r}} ~\tfrac{1}{2} \|\mathcal{A}(UU^\dagger) - y\|_2^2.
\end{equation}

Note that the (convex) PSD constraint $\rho \succeq 0$ and the (non-convex) rank constraint $\texttt{rank}(\rho) \leq r$ are automatically satisfied through the factorization. 

In [1]:
import sys
import os 
import warnings
import numpy as np

sys.path.append('../qutomo') 
warnings.filterwarnings("ignore")


In [2]:
import states
import projectors
import measurements
import methods

## Qubits

We are going to reconstruct a 3-qubit state: GHZ(3).

In [3]:
num_qubits = 3

## Labels

Let's generate some label combinations for the number we are given in order to start things moving. Each label combination represents a measurement circuit applied to the output of the quantum gates implementing the state we want to reconstruct. 

So there are `4^(num_labels)` possible labels, since any of the 4 Pauli operators `X`, `Y`, `Z`, `I` can be applied on each qubit. This means there are `4^3=64` possible labels for the case of 3-qubit quantum state.

We utilize `50` out of the `64` possible measurement circuits in the reconstruction.

In [4]:
num_labels = 50
labels     = projectors.generate_random_label_list(num_labels, num_qubits)
np.array(labels)

array(['ZZY', 'XZZ', 'ZYZ', 'ZXX', 'ZIZ', 'XXI', 'IXZ', 'IYI', 'IXY',
       'YXY', 'ZIY', 'YYX', 'XXY', 'IZY', 'XYI', 'ZXY', 'YIZ', 'XIX',
       'YXX', 'IXI', 'YZY', 'YIY', 'XYX', 'IIZ', 'YZI', 'XZI', 'IIX',
       'IZI', 'YXI', 'XII', 'ZYY', 'IZX', 'YYZ', 'XIY', 'IIY', 'ZYX',
       'YZX', 'YIX', 'ZZZ', 'XXZ', 'ZII', 'ZZI', 'ZYI', 'ZXZ', 'IZZ',
       'YZZ', 'XXX', 'XZX', 'IYY', 'XYY'], dtype='<U3')

## Circuits

Now assuming a base GHZ circuit (state) over `num_qubits`, we now build and apply to it the measurement circuits corresponding to the labels above and then collect the measurement results. Each measurement circuit is applied `num_shots=100` times to the prepaperd quantum state. Measurements for each label, is the number of counts (out of `num_shots`) each of the `2^(num_qubits)` possible outputs (binary vectors) appear. In particular, each of the 'X', 'Y', 'Z' operators applied to a qubit can render 1 or -1 (encoded in binary as 0 or 1) as output. We use `qiskit` routines for this internally.

In [5]:
backend    = 'qasm_simulator'
num_shots  = 100

state = states.GHZState(num_qubits)
state.create_circuit()
data_dict_list = state.execute_measurement_circuits(labels, 
                                                    backend=backend, 
                                                    num_shots=num_shots)

This is how one of the measurement looks like for one of the labels:

In [6]:
data_dict_list[2]

{'measurement_circuit_name': 'GHZ-ZYZ',
 'circuit_name': 'GHZ',
 'label': 'ZYZ',
 'count_dict': {'000': 18, '111': 29, '010': 27, '101': 26},
 'backend': 'qasm_simulator',
 'num_shots': 100}

## Measurements

Count information from the Qiskit experiment are now organized in a `<label> : <dictionary-of-counts>` dictionary and fed into a MeasurementStore abstraction for storage. This can save count information in a list of `.pickle` files under a given directory (default option).

In [7]:
measurement_dict = {item['label'] : item['count_dict'] for item in data_dict_list}

In [8]:
measurement_store_path = 'data/measurements'
if not os.path.exists(measurement_store_path):
    os.makedirs(measurement_store_path)

In [9]:
measurement_store = measurements.MeasurementStore(measurement_dict)
measurement_store.save(measurement_store_path)

## `Projectors`: Pauli string operators

The labels are associated with Kronecker products of corresponding Pauli matrices (in the "Pauli correlation measurements" settings which is our focus). The resulting (sparse) matrices and their application logic (to quantum state vectors) is captured in storage-and-compute-efficient "Projector" objects: these are in fact "Pauli string" operators since they are not actually "projectors" in the mathematical sense. 

In [10]:
projector_store_path   = 'data/projectors'
if not os.path.exists(projector_store_path):
    os.makedirs(projector_store_path)

In [11]:
projector_store = projectors.ProjectorStore(labels)
projector_store.populate(projector_store_path)

## State reconstruction using MiFGD in `qutomo`

We now run `MiFGD` algorithm as implemented in `qutomo`. 

To solve 

\begin{equation}\label{eq:factobj}
\min_{U \in \mathbb{C}^{d \times r}} ~\tfrac{1}{2} \|\mathcal{A}(UU^\dagger) - y\|_2^2,
\end{equation}

`MiFGD` iterates as follows:

\begin{align}
U_{i+1} &= Z_{i} - \eta \mathcal{A}^\dagger \left(\mathcal{A}(Z_i Z_i^\dagger) - y\right) \cdot Z_i, \label{eq:MiFGD1}\\
Z_{i+1} &= U_{i+1} + \mu \left(U_{i+1} - U_i\right), \label{eq:MiFGD2}
\end{align}

where $\eta$ is the step size, and $\mu$ is the momentum parameter. `MiFGD` essentially applies Nesterov's acceleration technique to factored gradient descent:

\begin{align}
U_{i+1} &= U_{i} - \eta \nabla f(U_i U_i^\dagger) \cdot U_i \\ &= U_{i} - \eta \mathcal{A}^\dagger \left(\mathcal{A}(U_i U_i^\dagger) - y\right) \cdot U_i.
\end{align}


This amounts to instantiating a `Worker` object - passing a dictionary to the constructor - and then inspecting its attributes, its `state` attribute in particular. We do not use acceleration in the example that follows. 

In [12]:
params_dict = {'measurement_store_path' : measurement_store_path,
               'projector_store_path'   : projector_store_path,
               'num_iterations'         : 1000,
               'eta'                    : 0.01,
               'mu'                     : 3/4,
               'n'                      : num_qubits,
               'num_labels'             : num_labels,
               'backend'                : backend,
               'num_shots'              : num_shots}

worker = methods.BasicWorker(params_dict)
worker.compute()

Loading projectors
59 projectors loaded
Projectors ready to compute with
Loading measurements
59 measurements loaded
Measurements ready to compute with
0


In [13]:
reconstructed_state = worker.state
reconstructed_state

array([ 0.50043795+0.46604736j, -0.02031182+0.04987667j,
        0.03092972-0.01444688j, -0.01296224+0.0030353j ,
        0.03284134+0.0059158j , -0.00111053-0.0214324j ,
       -0.02987869+0.02364279j,  0.5324162 +0.46370628j])

In [14]:
reconstructed_density_matrix = np.outer(reconstructed_state, reconstructed_state.T.conj())

What about the density_matrix we are trying to get to? Actually we can also get if from the circuit we set out to work with (see previously):

In [15]:
target = states.GHZState(num_qubits)
target.create_circuit()
target_density_matrix = target.get_state_matrix()

/Users/jlylekim/opt/anaconda3/envs/mifgd/lib/python3.8/site-packages/qiskit/tools/qi/qi.py:355: DeprecationWarning: This function is deprecated and will be removed in a future release. Please use `numpy.outer` function for similar functionality.
  warnings.warn(


Are the two states close to each other? Let's import the fidelity routine from `qiskit` and apply it

In [16]:
from qiskit.quantum_info import state_fidelity

In [17]:
state_fidelity(target_density_matrix,
               reconstructed_density_matrix,
               validate=False)

0.9656147717883425